In [1]:
import requests
import bs4
import time
import pandas as pd
import re

In [3]:
# FILENAME='verbs.csv'

In [4]:
# verbs = pd.read_csv('hebrew_verbs.csv', sep = ';')

In [5]:
error_msg='BANNED'
# pause = 0.1

In [8]:
# links = []

In [9]:
# n = 0
# for verb in verbs['Verb']:
#     if str(verb) != 'nan':
#         n += 1
#         req = requests.get('https://www.pealim.com/search/?q='+verb)
#         if req.status_code == 521:
#             print(error_msg)
#             print(n)
#             break
# #         time.sleep(pause)
#         parser = bs4.BeautifulSoup(req.text, 'lxml')
#         div = parser.find('div',attrs={'class' : 'verb-search-lemma'})
#         if div:
#             for a in div.children:
#                 link = a.get('href')
#                 if link:
#                     links.append(link)
#                 else:
#                     print('no link - ' + verb)
#         else:
#             print(verb)

ללעמוד
להשתנה
להתאחדים
להצתייעץ


In [10]:
# links = pd.DataFrame(links, columns=['Link'])
# links.to_csv('links.csv')

In [111]:
links = pd.read_csv('links.csv', sep = ',')

In [112]:
cookies = {'showmeaning':'off','translit':'isr','hebstyle':'bp'}

In [12]:
# vowel_pattern = re.compile(r"[\u0591-\u05C7]")
# def remove_vowels(text: str) -> str:
#     return re.sub(vowel_pattern, "", text)

In [113]:
res = []

In [114]:
for link in links['Link']:
    req = requests.get('https://www.pealim.com' + link, cookies=cookies)
    if req.status_code == 521:
        print(error_msg)
        print(link)
        break
    verb_description = []
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    p = parser.find('b')
    verb_description.append('https://www.pealim.com' + link)
    verb_description.append(p.text)
    div = parser.find('div',attrs={'class' : 'lead'})
    verb_description.append(div.text)
    tds =  parser.findAll('td',attrs={'class' : 'conj-td'})
    for td in tds:
        div = td.find('span',attrs={'class' : 'menukad'})
        verb_description.append(div.text)
    res.append(verb_description)
    

In [115]:
res = pd.DataFrame(res)
res.to_csv('res.csv',sep = ';')

In [116]:
res2 = pd.read_csv('res.csv', sep = ';')